In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_excel('Hate Speech Roman Urdu (HS-RU-20).xlsx')

In [5]:
df.head()

,Sentence,Neutral (N) / Hostile (H)
0,kya mein bhooka hon? kutia ab tum ney ye pooch...,H
1,"khawateen ghaas ki tarah hain, inhen baqaidagi...",H
2,aik aurat ke tor par aap ko apne ghar ki safai...,N
3,Afghani dehshat gard hotay hain,H
4,tamam sarkari hukkaam chor hain,H


Artificial Neural Network

In [58]:
from sklearn.model_selection import train_test_split
sentences=df['Sentence'].values
y=df['Neutral (N) / Hostile (H)'].values
sentences_train,sentences_test,y_train,y_test=train_test_split(sentences,y,test_size=0.25,random_state=10000)

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)




In [56]:
from keras.models import Sequential
from keras import layers
input_dim=X_train.shape[1]
model=Sequential()
model.add(layers.Dense(10,input_dim=input_dim,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))


In [60]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                72250     
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 72,261
Trainable params: 72,261
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train,epochs=100,validation_data=(X_test,y_test),batch_size=10)


In [57]:
from keras.backend import clear_session
clear_session()

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Convolutional Neural Network

In [62]:
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=8441)
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  
print(sentences_train[5])
print(X_train[5])

kashmir mein aaj koi bhi modi ka naam lewa nahi hai
[37, 4, 33, 36, 17, 49, 7, 133, 3372, 13, 1]


In [63]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 8441
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [66]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 8441, 100)         725000    
                                                                 
 conv1d (Conv1D)             (None, 8437, 128)         64128     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 790,429
Trainable params: 790,429
Non-trainable params: 0
________________________________________________

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))